In [1]:
import json
from pathlib import Path

from vu_models import QUESTIONS_PER_FOLDER, Question, Topics

from dreamai.ai import ModelName

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# data_dir = Path("/media/hamza/data2/MATH/train/")
# questions_dir = Path("math_102_questions")
# question_id = 0
# for folder in data_dir.iterdir():
#     if folder.is_dir() and folder.name != "counting_and_probability":
#         folder_questions = []
#         for question_file in folder.glob("*.json"):
#             question = json.loads(question_file.read_text())
#             if "5" in question["level"]:
#                 dest = questions_dir / f"{folder.name}/{question_id}.json"
#                 os.makedirs(dest.parent, exist_ok=True)
#                 with open(dest, "w") as f:
#                     json.dump(
#                         {
#                             "id": str(question_id),
#                             "problem": question["problem"],
#                             "solution": question["solution"],
#                         },
#                         f,
#                         indent=2,
#                     )
#                 question_id += 1

In [3]:
# created_topics = [
#     create_topic(topic, model=ModelName.HAIKU, attempts=2)
#     for topic in math_102_text.splitlines()
# ]

In [4]:
# topics = Topics(
#     topics={
#         topic.name: Topic(**topic.model_dump()) for topic in created_topics if topic
#     }
# )

In [5]:
# with open("math_102_may_1_topics.json", "w") as f:
#     json.dump(topics.model_dump(), f, indent=2)

In [6]:
questions_dir = Path("math_102_questions")
questions = {
    json.loads(question_file.read_text())["id"]: Question.model_construct(
        **json.loads(question_file.read_text())
    )
    for folder in questions_dir.iterdir()
    for question_file in list(folder.glob("*.json"))[:QUESTIONS_PER_FOLDER]
}

In [7]:
DAY = 5
valid_questons_file = f"math_102_may_{DAY}_questions.json"
current_topcs_file = f"math_102_may_{DAY}_topics.json"

In [8]:
topics = Topics(**json.load(open(current_topcs_file, "r")))

In [9]:
valid_questions = {
    id: Question.model_construct(**question)
    for id, question in json.load(open(valid_questons_file)).items()
}

In [10]:
counter = 0
n_questions = 2
for id, question in questions.items():
    if id not in valid_questions and counter < n_questions:
        valid_question = Question.model_validate(
            question.model_copy(deep=True),
            context={
                "topics": topics,
                "questions": valid_questions,
                "ask_kwargs": {"model": ModelName.GPT_4, "attempts": 3},
            },
        )
        counter += 1


ASSIGNING GROUP for 804


UPDATING CONCEPT QUESTIONS for 804

QUESTION ID: 804

CREATING SUBQUESTIONS for 804


VALIDATING ID for 804_1


ASSIGNING GROUP for 804_1


UPDATING CONCEPT QUESTIONS for 804_1

QUESTION ID: 804_1

CREATING SUBQUESTIONS for 804_1


VALIDATING ID for 804_1_1


ASSIGNING GROUP for 804_1_1


UPDATING CONCEPT QUESTIONS for 804_1_1

QUESTION ID: 804_1_1

CREATING SUBQUESTIONS for 804_1_1


VALIDATING ID for 804_1_2


ASSIGNING GROUP for 804_1_2


UPDATING CONCEPT QUESTIONS for 804_1_2

QUESTION ID: 804_1_2

CREATING SUBQUESTIONS for 804_1_2


NUMBER OF SUBQUESTIONS: 2


VALIDATING ID for 804_2


ASSIGNING GROUP for 804_2


UPDATING CONCEPT QUESTIONS for 804_2

QUESTION ID: 804_2

CREATING SUBQUESTIONS for 804_2


VALIDATING ID for 804_2_1


ASSIGNING GROUP for 804_2_1


UPDATING CONCEPT QUESTIONS for 804_2_1

QUESTION ID: 804_2_1

CREATING SUBQUESTIONS for 804_2_1


VALIDATING ID for 804_2_2


ASSIGNING GROUP for 804_2_2


UPDATING CONCEPT QUESTIONS for 804_2_2

Q

In [11]:
with open(f"math_102_may_{DAY}_topics.json", "w") as f:
    json.dump(topics.model_dump(), f, indent=2)

In [12]:
with open(f"math_102_may_{DAY}_questions.json", "w") as f:
    json.dump(
        {id: {**question.all_dump()} for id, question in valid_questions.items()},
        f,
        indent=2,
    )

In [13]:
# pdf_file = "/media/hamza/data2/algebra.pdf"
# collection_name = "algebra_collection"

In [14]:
# pdf_collection = pdf_to_collection(
#     pdf_file,
#     collection_name=collection_name,
#     chunk_size=4000,
#     chunk_overlap=200,
#     device="cuda",
#     delete_existing=True,
# )

In [15]:
# pdf_collection = chroma_collection(name=collection_name, delete_existing=False)
# pdf_collection.count()

In [18]:
None in [1,2]

False